# All-site data extraction from Google Analytics

A possible (and first choice) success metric for the Academy posts is looking at which posts led to a conversion – this is the Goal Completion data. We'll have to walk back from the all-site data, as the conversion are not present in the Academy-only view.

In [ ]:
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

# There was an error when trying to reuse the functions from
# 03_GA_page_sessions, which is why they are all rewritten in
# this notebook. Will troubleshoot after MVP
from ga_extractor import ga_to_df

import pandas as pd
import numpy as np
import json

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

# 1. Checking that the connection is setup

In [ ]:
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = 'keys/all_site_api.json'
VIEW_ID = 'ga:151086072'

In [ ]:
def initialize_analyticsreporting():
    """Initializes an Analytics Reporting API V4 service object.
    Returns: An authorized Analytics Reporting API V4 service object."""

    credentials = ServiceAccountCredentials.from_json_keyfile_name(KEY_FILE_LOCATION, SCOPES)

    # Build the service object.
    analytics = build('analyticsreporting', 'v4', credentials=credentials)

    return analytics

In [ ]:
def get_report(analytics, metrics, dimensions, start, end):
    """Queries the Analytics Reporting API V4.
    Args:
    analytics: An authorized Analytics Reporting API V4 service object.
    Returns:
    The Analytics Reporting API V4 response.
    Using date range 2020-11-09 to 2020-11-15 for testing purposes"""
    METS = [f'ga:{metric}' for metric in metrics]
    DIMS = [f'ga:{dimension}' for dimension in dimensions]
    
    return analytics.reports().batchGet(
        body={
            'reportRequests': [
                                {
                                    'viewId': VIEW_ID,
                                    'dateRanges': [{'startDate': start
                                                    , 'endDate': end}],
                                    'metrics': [{'expression': expression} for expression in METS],
                                    'orderBys': [{'fieldName': METS[0], 
                                                  'sortOrder': 'DESCENDING'}],
                                    'dimensions': [{'name': name} for name in DIMS]
                                }]
            }).execute(), METS, DIMS

In [ ]:
def to_df(response, METS, DIMS):
    data_dict = {f"{i}": [] for i in DIMS + METS}
    
    for report in response.get('reports', []):
        rows = report.get('data', {}).get('rows', [])
        for row in rows:
            for i, key in enumerate(DIMS):
                data_dict[key].append(row.get('dimensions', [])[i])
            date_values = row.get('metrics', [])
            for values in date_values:
                all_values = values.get('values', [])
                for i, key in enumerate(METS):
                    data_dict[key].append(all_values[i])
                    
    df = pd.DataFrame(data=data_dict)
    df.columns = [col.split(':')[-1] for col in df.columns]
    
    return df

In [ ]:
def ga_to_df(metrics,dimensions,start,end):
    '''metrics = list, dimesnions = list,
    start = str, end = str'''
    analytics = initialize_analyticsreporting()
    response, METS, DIMS = get_report(analytics, metrics, dimensions, start, end)
    response_df = to_df(response, metrics, dimensions)
    
    return response_df

In [ ]:
# Goal 12: book a demo success


traceback_goal12 = ga_to_df(metrics=['goal12Completions'], 
                            dimensions=['goalPreviousStep1',
                                        'goalPreviousStep2',
                                        'goalPreviousStep3'],
                            start='2020-01-01',
                            end='2020-11-30')

In [ ]:
traceback_goal12.head()

In [ ]:
# Goal 13: seen demo page


traceback_goal13 = ga_to_df(metrics=['goal13Completions'], 
                            dimensions=['goalPreviousStep1',
                                        'goalPreviousStep2',
                                        'goalPreviousStep3'],
                            start='2020-01-01',
                            end='2020-11-30')

In [ ]:
traceback_goal13.head()

# Combine goal12 and goal13 for MVP

- extract rows that refer to the academy, egal whether step 1, 2 or 3
- combine 2 dataframes
- cols = post_url | goal12 | goal13
- prepend 'https://plana.earth' to urls

# 2.1 extract academy rows for Goal 12

In [ ]:
url = r'(\/academy\/[a-z])'

In [ ]:
# only keep academy articles

df_12 = traceback_goal12[(traceback_goal12['goalPreviousStep2'].str
                                                            .contains(url) == True) | 
                      (traceback_goal12['goalPreviousStep3'].str
                                                            .contains(url) == True)]

In [ ]:
df_12

In [ ]:
# need to drop 23 and 24

df_12 = df_12.drop([23, 24])

In [ ]:
df_12

In [ ]:
df_12 = df_12.drop(['goalPreviousStep1'], axis=1)

In [ ]:
# replacing unwanted values
unwanted = ['/', '/academy/']

df_12 = df_12.replace(to_replace=unwanted, value=np.nan)

In [ ]:
df_12

In [ ]:
# new_col of col1 values
# fillna() with prev step vals if NaN

df_12.goalPreviousStep2.fillna(df_12.goalPreviousStep3, inplace=True)

In [ ]:
df_12

In [ ]:
df_12 = df_12.drop(['goalPreviousStep3'], axis=1)

In [ ]:
df_12

In [ ]:
df_12.info()

In [ ]:
# conveting goalcompletions to int64

df_12['goal12Completions'] = pd.to_numeric(df_12.goal12Completions)

In [ ]:
# groupby article name

df_12_group = df_12.groupby('goalPreviousStep2', as_index=False).agg({'goal12Completions':'sum'})

In [ ]:
df_12_group

# 2.2 extract academy rows for Goal 13

In [ ]:
traceback_goal13

In [ ]:
# url = r'(\/academy\/[a-z])'
# only keep academy articles

df_13 = traceback_goal13[(traceback_goal13['goalPreviousStep1'].str.contains(url) == True) | 
                         (traceback_goal13['goalPreviousStep2'].str.contains(url) == True) | 
                         (traceback_goal13['goalPreviousStep3'].str.contains(url) == True)]

In [ ]:
df_13

In [ ]:
# removing manually, will set up regex finder at later stage

unwanted_13 = ['/', '/academy/', '/about', '/faq', '/offsetting', 
               '/academy/how-to-check-the-sustainability-of-your-company/?hss_channel=tw-805839858992316416', 
               '/project/ocean-plastic-collection', '/projects', '(entrance)', 
               '/academy/is-it-too-late-for-our-planet/?preview_id=273&preview_nonce=80c11f0354&preview=true&_thumbnail_id=3911', 
               '/academy/the-benefits-of-monitoring-carbon-emissions-for-a-business/?hss_channel=lcp-17966020', 
               '/academy/the-benefits-of-monitoring-carbon-emissions-for-a-business/facebook-sharing-3/', 
               '/project/solid-waste-management-and-transformation-from-waste-to-revenue-in-gasabo-district', 
               '/academy/author/tara/', '/academy/?s=greenwashing&post_type=post', 
               '/academy/why-measure-carbon-emissions/wnitqlvwxbrd1p5kh0xnn3bwynvt0ybrkrv_mlppqba5bdc65pesfgco_azlypy1bs6jarbtw7xz2xsmkmv3ckkl2sqhxzg25tcybiqu4ikdh5qpqsmu_ego1e5gmnn8anhzpoyf-2/', 
               '(not set)' , '/academy/?s=whitepaper&post_type=post', 
               '/academy/release-carbon-manager-software/?fbclid=IwAR3nKFCmA8_dqWvoiVJqrMF17hAGS9aR9sfPfq-FXPwahG9Nzkv-TOxq3YU', 
               '/academy/release-carbon-manager-software/?hss_channel=lcp-17966020', '/academy/tag/spotify/', 
               '/academy/category/forests/', 
               '/academy/is-it-too-late-for-our-planet/?preview_id=273&preview_nonce=80c11f0354&preview=true&_thumbnail_id=3911', 
               '/academy/category/waste/', '/imprint',
               '/academy/top-options-reduce-carbon-footprint/#35&utm_content=Planetarium+#35&utm_medium=email_action&utm_source=customer.io', 
               '/academy/why-measure-carbon-emissions/https:/plana.earth/academy/why-measure-carbon-emissions/']



df_13 = df_13.replace(to_replace=unwanted_13, value=np.nan)

In [ ]:
df_13

In [ ]:
# filling NaN in goalPreviousStep1 with prev step goalPreviousStep2

df_13.goalPreviousStep1.fillna(df_13.goalPreviousStep2, inplace=True)

In [ ]:
df_13

In [ ]:
# Now filling NaN in goalPreviousStep1 with prev step goalPreviousStep3

df_13.goalPreviousStep1.fillna(df_13.goalPreviousStep3, inplace=True)

In [ ]:
df_13

In [ ]:
# dropping unneeded cols goalPreviousStep2 and goalPreviousStep3
df_13 = df_13.drop(['goalPreviousStep2','goalPreviousStep3'], axis=1)

In [ ]:
df_13

In [ ]:
# keeping rows where goalstep1 not NaN

df_13 = df_13[df_13['goalPreviousStep1'].notna()]

In [ ]:
df_13

In [ ]:
# conveting goalcompletions to int64

df_13['goal13Completions'] = pd.to_numeric(df_13.goal13Completions)

In [ ]:
df_13_group = df_13.groupby('goalPreviousStep1', as_index=False).agg({'goal13Completions':'sum'})

In [ ]:
df_13_group

# 3. Merging df_12 and df_13

In [ ]:
df_12_group

In [ ]:
df_goals = pd.concat([df_12_group, df_13_group]).reset_index()

In [ ]:
df_goals = df_goals.drop(['index'], axis=1)

In [ ]:
df_goals

In [ ]:
df_goals.goalPreviousStep2.fillna(df_goals.goalPreviousStep1, inplace=True)

In [ ]:
df_goals

In [ ]:
df_goals = df_goals.drop(['goalPreviousStep1'], axis=1)

In [ ]:
df_goals

In [ ]:
df_goals.fillna(0, inplace=True)

In [ ]:
df_goals

In [ ]:
df_goals = df_goals.groupby('goalPreviousStep2', as_index=False).agg({'goal12Completions':'sum',
                                                                      'goal13Completions':'sum'})

In [ ]:
df_goals

In [ ]:
# adding 'https://plana.earth' prefix to academy_url

df_goals['goalPreviousStep2'] = 'https://plana.earth' + df_goals['goalPreviousStep2'].astype(str)

In [ ]:
df_goals

In [ ]:
goal_completions = df_goals.rename(columns={'goalPreviousStep2':'academy_url'})

In [ ]:
goal_completions

In [ ]:
#import pickle

#with open('../01_LDA_model/pkl_cellar/goal_completions.pkl', 'wb') as gc:
#    pickle.dump(goal_completions, gc, protocol=pickle.HIGHEST_PROTOCOL)